# Infrastructure Setup

In [ ]:
import os
from chi import context

# We select which chameleon site we are using and choose the project
context.use_site('CHI@UC')
context.choose_project()
username = os.environ.get('USER').replace("_", "-")

## Creating a 1-day lease

In [ ]:
from chi import lease
from datetime import timedelta

In [ ]:
node_type="compute_cascadelake_r"
instance_count = 3

In [ ]:
mpi_lease = lease.Lease(f"{username}-mpi-lease", duration=timedelta(days=4))

mpi_lease.add_node_reservation(amount=instance_count, node_type=node_type)
# Register a storage network which is required to use NFS Shares
mpi_lease.add_network_reservation(network_name=f"{username}-storage-network", usage_type='storage')
mpi_lease.add_fip_reservation(1)
mpi_lease.submit(idempotent=True)

## Creating server instances

In [ ]:
from chi import server

In [ ]:
mpi_master=server.Server(
        f"{username}-mpi-master",
        reservation_id=mpi_lease.node_reservations[0]["id"],
        image_name="Ubuntu22.04-HPC-MPI-Spack",
        network_name=f"{username}-storage-network"
)
mpi_master_hostname = mpi_master.name
mpi_master.submit(idempotent=True)

mpi_workers = []
mpi_worker_hostnames = []
for i in range(instance_count - 1):
    mpi_worker=server.Server(
            f"{username}-mpi-worker-{i+1}",
            reservation_id=mpi_lease.node_reservations[0]["id"],
            image_name="Ubuntu22.04-HPC-MPI-Spack",
            network_name=f"{username}-storage-network"
    )
    mpi_worker.submit(idempotent=True)
    mpi_workers.append(mpi_worker)
    mpi_worker_hostnames.append(mpi_worker.name)

In [ ]:
all_hostnames = mpi_master_hostname + "," + ",".join(mpi_worker_hostnames)

In [ ]:
fip = mpi_lease.get_reserved_floating_ips()[0]
server.associate_floating_ip(mpi_master.id, fip)

## Create a New Share

In [ ]:
from chi import share

In [ ]:
share_names = [s.name for s in share.list_shares()]

In [ ]:
share_name = f"{username}-share"
if share_name not in share_names:
    print(f"Creating new share - {share_name}")
    mpi_share = share.create_share(size=1, name=f"{username}-share")
else:
    mpi_share = share.get_share(share_name)

In [ ]:
nfs_share = share.get_share(mpi_share.id)
nfs_share

In [ ]:
nfs_share_export_locations = nfs_share.export_locations[0]
nfs_share_export_locations

## Create inventory.ini to work with ansible

In [ ]:
with open("/work/inventory.ini", "w") as f:
    f.write("[master_node]\n")
    f.write(f"{mpi_master.name} ansible_host={fip}\n\n")
    
    f.write("[worker_nodes]\n")
    f.write("\n".join(f"{w.name} ansible_host={w.addresses[f"{username}-storage-network"][0]['addr']}" for w in mpi_workers))
    f.write("\n\n")
    f.write("[worker_nodes:vars]\n")
    f.write(f"ansible_ssh_common_args='-o ProxyJump=cc@{fip}'")

## Use Ansible to create an MPI Cluster

In [ ]:
nfs_shares = [
    {"export": nfs_share_export_locations, "mount": "/mnt/share1"}
]

In [ ]:
import ansible_runner
import tempfile
tmpdir = tempfile.TemporaryDirectory()
ansible_run = ansible_runner.run(
    private_data_dir=tmpdir.name,
    inventory="/work/inventory.ini",
    envvars = {
        "ANSIBLE_CONFIG": f"/work/ansible.cfg"
    },
    extravars={
        "nfs_shares": nfs_shares
    },
    playbook=f"/work/mpi-cluster.yml",
    verbosity=0
)

In [ ]:
mpi_master.upload("/work/examples/src/hello.c", "/home/cc/hello.c")
mpi_master.upload("/work/examples/mpi_jobs/run_hello_nfs.sh", "/home/cc/run_hello.sh")

In [ ]:
mpi_master.execute(f'bash -lc "source ./run_hello.sh {mpi_master_hostname} {",".join(mpi_worker_hostnames)}"')